import gzip
import json
import re
path = 'jawiki-country.json.gz'
wiki_jsons = [json.loads(line) for line in gzip.open(path, 'r').readlines()]
text = list(filter(lambda x: x['title'] == 'イギリス', wiki_jsons))[0]['text']
reg = re.compile(r'(?P<main>\{\{基礎情報(?:[^{}]+|\{\{[^{}]*?\}\})*\}\})'
                                  # (.*)\}\})'
                 , re.DOTALL)
result = reg.search(text)
template_dict = {}
for i, text in enumerate(result.group('main').strip('{}').split('\n|')):
    k, v = re.split(r'\s=\s|\s', text, 1)
    print(k, ':', v)
    template_dict[k] = v

In [4]:
reg = re.compile(r'\'+(?P<text>[^\']+)\'+')
{k: reg.sub(r'\1', v) for k, v in template_dict.items()}
# print('\n'.join(['{0}: {1}'.format(k, v) for k, v in template_dict.items()]))

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO': '3166-1 = GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': '[[エリザベス2世]]'

In [5]:
reg = re.compile(r'(?:\[\[([^\[\]\|]+\|?)+\]\])|(?:\'+(?P<text>[^\']+)\'+)')
{k: reg.sub(r'\1', v) for k, v in template_dict.items()}
# print('\n'.join(['{0}: {1}'.format(k, v) for k, v in template_dict.items()]))

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO': '3166-1 = GB / GBR',
 'ccTLD': '.uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': 'エリザベス2世',
 '元首等肩書': 

In [6]:
def replace_word(targets):
    print(targets)
    print(targets.group())
    r = [t for t in targets.groups() if t]
    if len(r):
        return r[0]
    return ''

rep_dic = {r'(?P<a><ref>.*?</ref>)': None,
           r'(?P<b><ref\s.+?/>)': None,
           r'(?P<c><ref.*?</ref>)': None,
           r'(?P<d><br\s?/>)': None,
           r'(?:\{\{(?P<hige>[^{}\|]+\|?)+\}\})': 'hige',
           r'(?:\[\[(?P<hoge>[^\[\]\|]+\|?)+\]\])': 'hoge',
           r'(?:\'+(?P<text>[^\']+)\'+)': 'text',
           r'(?:^\*+(?P<hage>.+))': 'hage'}

def replace_dict(match):
    for key, value in rep_dic.items():
        hoge = re.compile(key)
        if hoge.match(match.group(0)) is not None:
            if value is None:
                return ''
            return match.group(value)
# '\[\[([^\[\]\|]+\|?)+\]\]'
# use these three lines to do the replacement
pattern = re.compile(r"|".join(rep_dic.keys()), re.MULTILINE | re.DOTALL)
# print(pattern)

template_dict = {k: pattern.sub(replace_dict, v) for k, v in template_dict.items()}
# reg = re.compile(r'(?:<ref>.*?</ref>)|(?:<ref\s.+?/>)|(?:<ref.*?</ref>)|(?:<br\s?/>)|(?:\{\{?P<hige>([^{}\|]+\|?)+\}\})|(?:\[\[(?P<hoge>[^\[\]\|]+\|?)+\]\])|(?:\'+(?P<text>[^\']+)\'+)', re.MULTILINE | re.DOTALL)
# {k: reg.findall(v) for k, v in template_dict.items()}
# {k: reg.sub(replace_word, v) for k, v in template_dict.items()}

In [7]:
template_dict['国旗画像']

'Flag of the United Kingdom.svg'